# Results Machine Learning Scikit-Learn
Ultimately, we want to see which variables have the greatest impact on AQI. To do this we must perform a machine learning analysis and create a prediction algorithm. 


In [ ]:
#| label: import-packages
#| echo: false
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

First, missing data must be addressed. This can be done in a variety of ways, but for this we replace NA values with the mean for the respective city and group by week. If there is no data to take a mean of, that row will be dropped. 


In [ ]:
#| echo: false
#| label: read-csv
df = pd.read_csv('https://raw.githubusercontent.com/wu-msds-capstones/Air-Quality-Index/main/data/metro_1mil.csv')
len(df)

In [ ]:
#| label: check-isna
#| echo: false
df.isna().sum()

In [ ]:
#| label: met-dataframe
#| echo: false
met = df[[
    'date',
    'state',
    'county',
    'city',
    'population',
    'density',
    'aqi',
    'category',
    'mean_temperature_fahrenheit',
    'mean_pressure_millibars',
    'mean_humidity_percent_relative_humidity',
    'mean_wind_knots',
    'mean_co_ppm',
    'mean_no2_ppb',
    'mean_ozone_ppm',
    'mean_so2_ppb',
    'mean_pm100_micrograms_per_cubic_meter',
    'mean_pm25_micrograms_per_cubic_meter',
    'mean_lead_micrograms_per_cubic_meter',
    'num_busses',
    'revenue',
    'operating_expense',
    'passenger_trips',
    'operating_hours',
    'passenger_miles',
    'operating_miles'
    ]]

met = met.rename(columns={'mean_temperature_fahrenheit': 'temp', 
                          'mean_pressure_millibars': 'pressure', 
                          'mean_humidity_percent_relative_humidity': 'humidity',
                          'mean_wind_knots': 'wind_speed', 
                          'mean_co_ppm': 'co', 
                          'mean_no2_ppb': 'no2',
                          'mean_ozone_ppm': 'o3', 
                          'mean_so2_ppb': 'so2', 
                          'mean_pm100_micrograms_per_cubic_meter': 'pm100',
                          'mean_pm25_micrograms_per_cubic_meter': 'pm25', 
                          'mean_lead_micrograms_per_cubic_meter': 'lead'})

In [ ]:
#| echo: false
#| label: check-NaN
met.isna().sum()

In [ ]:
#| echo: false
#| label: set-index-ts
met['date'] = pd.to_datetime(met['date'])
met.set_index('date', inplace=True)
#group by week, drop lead for too much missing data
met = met.groupby([pd.Grouper(freq='W'), 'state', 'county', 'city']).agg({
    'population': 'first',
    'density': 'first',
    'aqi': 'mean',
    'temp': 'mean',
    'pressure': 'mean',
    'humidity': 'mean',
    'wind_speed': 'mean',
    'co': 'mean',
    'no2': 'mean',
    'o3': 'mean',
    'so2': 'mean',
    'pm100': 'mean',
    'pm25': 'mean',
    'num_busses': 'mean',
    'revenue': 'mean',
    'operating_expense': 'mean',
    'passenger_trips': 'mean',
    'operating_hours': 'mean',
    'passenger_miles': 'mean',
    'operating_miles': 'mean'
}).reset_index()

Since AQI is the dependent variable being measured, all rows without AQI data are dropped. Certain cities have very little data and will be dropped out of necessity.


In [ ]:
#| echo: false
#| label: drop-missNaN-col
met = met[met['city'] != 'Virginia Beach']

In [ ]:
#| echo: false
#| label: drop-null-aqi
met = met.dropna(subset=['aqi'])

In [ ]:
#| echo: false
#| label: check-NaN-again
met.isna().sum()

The data collected has separate information for the city of New York City. NYC is divided into five boroughs, each within its own county. These values are grouped and averaged out to make NYC have the same amount of datapoints as every other city. This will also address data present in some New York boroughs but not others. Likewise, Kansas City spans two states and two counties, so those values are grouped together.


In [ ]:
#| echo: false
#| label: group-nyc-1-city
nyc = met[met['city'] == 'New York']
columns = ['date', 'aqi', 'temp', 'pressure','humidity',
           'wind_speed','co','no2','o3',
           'so2','pm100','pm25', 'num_busses',
           'revenue', 'operating_expense', 
           'passenger_trips', 'operating_hours', 
           'passenger_miles', 'operating_miles']

nyc = nyc[columns].groupby('date').mean().reset_index()
#Add data that was dropped
nyc['state'] = 'New York'
nyc['county'] = 'Multiple'
nyc['city'] = 'New York City'
nyc['population'] = 18908608
nyc['density'] = 11080.3

nyc = nyc[['date', 'state', 'county', 'city', 'population', 
     'density', 'aqi', 'temp', 'pressure','humidity',
     'wind_speed','co','no2','o3', 'so2','pm100',
     'pm25', 'num_busses', 'revenue', 'operating_expense', 
     'passenger_trips', 'operating_hours', 
     'passenger_miles', 'operating_miles']]

print(nyc)

In [ ]:
#| echo: false
#| label: replace-nyc-in-met
met = met[met['city'] != 'New York']
met = pd.concat([met, nyc], ignore_index=True)

In [ ]:
#| echo: false
#| label: dataframe-Kansas-City
#Do same with Kansas City
kc = met[met['city'] == 'Kansas City']
columns = ['date', 'aqi', 'temp', 'pressure','humidity',
           'wind_speed','co','no2','o3',
           'so2','pm100','pm25', 'num_busses',
           'revenue', 'operating_expense', 
           'passenger_trips', 'operating_hours', 
           'passenger_miles', 'operating_miles']

kc = kc[columns].groupby('date').mean().reset_index()

columns = ['temp', 'pressure','humidity',
           'wind_speed','co','no2','o3',
           'so2','pm100','pm25', 'num_busses',
           'revenue', 'operating_expense', 
           'passenger_trips', 'operating_hours', 
           'passenger_miles', 'operating_miles']
met[columns] = met[columns].fillna(met.groupby('city')[columns].transform('mean'))

kc['state'] = 'Missouri'
kc['county'] = 'Multiple'
kc['city'] = 'Kansas City'
kc['population'] = 1689556
kc['density'] = 620.7

kc = kc[['date', 'state', 'county', 'city', 'population', 
     'density', 'aqi', 'temp', 'pressure','humidity',
     'wind_speed','co','no2','o3', 'so2','pm100',
     'pm25', 'num_busses', 'revenue', 'operating_expense', 
     'passenger_trips', 'operating_hours', 
     'passenger_miles', 'operating_miles']]

met = met[met['city'] != 'Kansas City']
met = pd.concat([met, kc], ignore_index=True)

After this data cleaning, we are left with a total of eighteen cities across the country with a total metropolitan area population of greater than one million. The table below (figure 24324) shows all eighteen cities being used in the prediction algorithm and their respective row counts.


In [ ]:
#| echo: false
#| label: count-cities
ds = met.dropna()

ds.groupby('city').size().reset_index(name='count')

To perform a ML prediction algorithm, the predicted variable (AQI) must be discrete. To achieve this, we bin AQI data into discrete groups. Initially, we thought to bin them based on the existing AQI categories, but found that most of the data is grouped in the sub 100 range. Therefore, we expanded the bins, focusing the prediction on outcomes in the double digits. The bins chosen are: 

- 0-30
- 31-40
- 41-50 
- 51-60
- 61-70
- 71-80
- 81-90
- 91-100
- 101-150
- 151+


In [ ]:
#| echo: false
#| label: AQI-bin-ML
bins = [0, 30, 40, 50, 60, 70, 80, 90, 100, 150, 500]

labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
ds.loc[:,'aqi_discrete'] = pd.cut(ds['aqi'], bins=bins, labels=labels, right=True)

ds.head()

In [ ]:
#| echo: false
#| label: label-bin
labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
ds.loc[:,'aqi_discrete'] = pd.cut(ds['aqi'], bins=bins, labels=labels, right=True)

ds.head()

In [ ]:
#| echo: false
#| label: convert-year-month-datetime
ds.loc[:,'year'] = pd.to_datetime(ds['date']).dt.year
ds.loc[:,'month'] = pd.to_datetime(ds['date']).dt.month

A feature selector is run on the set of all variables. This chooses the best predictors of the dependent variable AQI.


In [ ]:
#| echo: false
#| label: aqi-X-y
#dependent variable aqi, all others (except aqi category) as independent variables
y = ds['aqi_discrete']
X = ds[['city',
        'population',
        'density',
        'temp',
        'pressure',
        'humidity',
        'wind_speed',
        'co',
        'no2',
        'o3',
        'so2',
        'pm100',
        'pm25',
        'num_busses', 
        'revenue', 
        'operating_expense', 
        'passenger_trips', 
        'operating_hours', 
        'passenger_miles', 
        'operating_miles',
        'year',
        'month'
        ]]

The following tools are used:

- Train Test Split
- One Hot Encoder
- Transformer
- Pipeline
- Standard Scaler 


In [ ]:
#| echo: false
#| label: split-train
#Split into train and test datasets, define model and encoder
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
encoder = OneHotEncoder(sparse_output=False, min_frequency=5, handle_unknown='infrequent_if_exist')

In [ ]:
#| echo: false
#| label: def-transformer
#define transformer with encoder and standardscaler

transformer = ColumnTransformer(
        [
            ('categories', encoder, ['city','year','month']),
            ('scaled_air_quality', StandardScaler(), [
                'population',
                'density',
                'temp',
                'pressure',
                'humidity',
                'wind_speed',
                'co',
                'no2',
                'o3',
                'so2',
                'pm100',
                'pm25',
                'num_busses', 
                'revenue', 
                'operating_expense', 
                'passenger_trips', 
                'operating_hours', 
                'passenger_miles', 
                'operating_miles'
                ]
            )
        ],
        remainder='drop', verbose_feature_names_out=False)
#fit transformer
transformer.fit(X_train, y_train)

Feature selection is done on the data. From this we are given the following variables: 

- City
- Temperature
- Humidity
- Carbon Monoxide
- Nitrogen Dioxide
- Ozone
- PM10
- PM2.5

These features are used in the final model.


In [ ]:
#| echo: false
#| #| label: feature-select
#feature selection
feature_selector = SelectKBest(k=10)
X_train_trans = transformer.transform(X_train)
X_train_trans_df = pd.DataFrame(
    X_train_trans, 
    columns = transformer.get_feature_names_out(),
    index = X_train.index)
feature_selector.fit(X_train_trans_df, y_train)
feature_selector.get_support()
feature_selector.scores_[feature_selector.get_support()]
X_train_trans_df.columns[feature_selector.get_support()]

In [ ]:
#| echo: false
#| #| label: use-features
#Use selected features
y = ds['aqi_discrete']
X = ds[[
    'city', 'temp', 'humidity', 'co', 'no2', 'o3', 'pm100', 'pm25'
    ]]
#Create train/test data with selected features
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train_enc = pd.DataFrame(encoder.fit_transform(X_train[['city']]),
                           columns = encoder.get_feature_names_out(), index = X_train.index)
X_train_trans = X_train[[
    'temp', 'humidity', 'co', 'no2','o3', 'pm100', 'pm25'
    ]].merge(X_train_enc, left_index = True, right_index = True)
X_test_enc = pd.DataFrame(encoder.fit_transform(X_test[['city']]),
                           columns = encoder.get_feature_names_out(), index = X_test.index)
X_test_trans = X_test[[
    'temp', 'humidity', 'co', 'no2','o3', 'pm100', 'pm25'
    ]].merge(X_test_enc, left_index = True, right_index = True)

With these features, we test various models with default parameters to see which is the most accurate at predicting AQI from this dataset. The five models tested are: 

- K nearest neighbors
- Tree model 
- Random Forest model
- Logistic Regression 
- Naive Bayes 

Running the models gives the output in figure 234324. Random Forest model is the most accurate and will be used for the final model.


In [ ]:
#| echo: false
#| output: false
#| label: model-select
model1 = KNeighborsClassifier(n_neighbors=5)
model2 = tree.DecisionTreeClassifier()
model3 = RandomForestClassifier(n_estimators=10, random_state=12)
model4 = LogisticRegression()
model5 = GaussianNB()

results = []

for model, label in zip([model1, model2, model3, model4, model5], ['KNN', 'Tree', 'Random Forest', 'Logistic', 'naive Bayes']):
    model.fit(X_train_trans, y_train)
    y_pred = model.predict(X_test_trans)
    model = label
    cohen_kappa = cohen_kappa_score(y_test, y_pred)
    accuracy = sum(y_test == y_pred)/len(y_test)
    
    results.append({
        'Model': label,
        'Cohen Kappa Score': cohen_kappa,
        'Accuracy': accuracy
    })

#display(pd.DataFrame(results))

![AirQuality Confusion Matrix 1](https://github.com/wu-msds-capstones/Air-Quality-Index/blob/main/images/AirQuality-ConfusionMatrix_init.png?raw=true)


In [ ]:
#| echo: false
#| #| label: def-best-model
#Create model from most accurate, RandomForest
model = RandomForestClassifier(n_estimators=10)
model.fit(X_train_trans, y_train)

#predict on test data
y_pred = model.predict(X_test_trans)

#Print results
print("cohen kappa score: ", cohen_kappa_score(y_test, y_pred))
print("accuracy: ", sum(y_test == y_pred)/len(y_test))
ConfusionMatrixDisplay(
    confusion_matrix = confusion_matrix(
        y_test, y_pred, 
        labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
        ), 
        display_labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
        ).plot(xticks_rotation='vertical')

In [ ]:
#| echo: false
#| label: improve-transformer
#redefined transformer
transformer = ColumnTransformer(
        [
            ('categories', encoder, ['city']),
            ('scaled_air_quality', StandardScaler(), [
                'temp',
                'humidity',
                'co',
                'no2',
                'o3',
                'pm100',
                'pm25'
                ]
            )
        ],
        remainder='drop', verbose_feature_names_out=False)
#Pipeline for simplification

classification_pipeline = Pipeline([('aqi_transformer', transformer),
                                    ('RF_model', RandomForestClassifier())
                                    ])
classification_pipeline.fit(X_train, y_train)

Hyperparameter optimization will be done to further improve the model. A randomized search is run with 100 iterations. The following hyperparameters are optimized:

- Max Categories
- Min Frequency
- Max Depth
- Max Features
- Min Samples Leaf
- Min Samples Split
- Num Estimators
- Bootstrap


In [ ]:
#| echo: false
#| label: hyper-parameters-optim
#hyperparameter optimization
classification_pipeline.get_params()

In [ ]:
#| echo: false
#| label: def-parameters
parameters = {
    'aqi_transformer__categories__max_categories': randint(3,30),
    'aqi_transformer__categories__min_frequency': randint(2,20),
    'RF_model__max_depth': randint(3, 30),
    'RF_model__max_features': [None, 'sqrt', 'log2'],
    'RF_model__min_samples_leaf': randint(2, 10),
    'RF_model__min_samples_split': randint(2, 10),
    'RF_model__n_estimators': randint(50, 200),
    'RF_model__bootstrap': [True, False]
}
n_iter_search = 10
random_search = RandomizedSearchCV(classification_pipeline, param_distributions=parameters,
                                   n_iter=n_iter_search, n_jobs=-1, cv=5)
random_search.fit(X_train, y_train)
print(random_search.best_score_)

Figure 242442 below shows the selected hyperparameters.


In [ ]:
#| echo: false
#| label: randsearch-get-params
random_search.best_params_
#random_search.best_estimator_.get_params()
params_df = pd.DataFrame(list(random_search.best_params_.items()), columns=['Parameter', 'Value'])

# Print the DataFrame
display(params_df)

Using these hyperparameters, we reach an accuracy of about 63%.


In [ ]:
#| echo: false
#| warning: false
#| label: predict-X_test
y_pred=random_search.predict(X_test)
print("Cohen Kappa Score: ", cohen_kappa_score(y_test, y_pred))
print("Accuracy: ", sum(y_pred == y_test)/len(y_test))

In [ ]:
#| echo: false
#| output: false
ConfusionMatrixDisplay(
    confusion_matrix = confusion_matrix(
        y_test, y_pred, 
        labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
        ), 
        display_labels = ['0-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100', '101-150', '151+']
        ).plot(xticks_rotation='vertical')

![AirQuality Confusion Matrix 2](https://github.com/wu-msds-capstones/Air-Quality-Index/blob/main/images/AirQuality-ConfusionMatrix.png?raw=true)

We can use this model for the prediction of AQI, exploring the features that predict it, and use that to generate conclusions for what particles to reduce or systems to increase.